DerivaML is a class library built on the Deriva Scientific Asset management system that is designed to help simplify a number of the basic operations associated with building and testing ML libraries based on common toolkits such as TensorFlow.  This notebook reviews the basic features of the DerivaML library.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import builtins
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
from deriva_ml.deriva_ml_base import ColumnDefinition, BuiltinTypes
from deriva_ml.schema_setup.test_catalog import create_test_catalog, DemoML
from deriva_ml.execution_configuration import ExecutionConfiguration, Workflow, Execution, WorkflowTerm, Term
from pathlib import Path
import pandas as pd
import tempfile

Set the details for the catalog we want and authenticate to the server if needed.

In [ ]:
hostname = 'dev.eye-ai.org'
domain_schema = 'demo-schema'

gnl = GlobusNativeLogin(host=hostname)
if gnl.is_logged_in([hostname]):
    print("You are already logged in.")
else:
    gnl.login([hostname], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")


Create a test catalog and get an instance of the DerivaML class.

In [ ]:
test_catalog = create_test_catalog(hostname, domain_schema)
ml_instance = DemoML(hostname, test_catalog.catalog_id)

In [ ]:
ml_instance.chaise_url("Subject")

In [ ]:
[t.name for t in ml_instance.find_features('Image')]

A feature is a set of values that are attached to a table in the DerivaML catalog. Instances of features are distingushed from one another by the ID of the execution that produced the feature value. The execution could be the result of a program, or it could be a manual process by which a person defines a set of values

To create a new feature, we need to know the name of the feature, the table to which it is attached, and the set of values that make up the feature.  The values could be terms from a controlled vocabulary, a set of one or more file based assets, or other values, such as integers, or strings. However, use of strings outside of controlled vocabularies is discouraged.

In [ ]:
# Prerequests for our feature, which will include a CV term and asset.

# Create a vocabulary and add a term to it to use in our features.
ml_instance.create_vocabulary("FeatureValue", "A vocab")
ml_instance.add_term("FeatureValue", "V1", description="A Feature Vale")

feature_asset = ml_instance.create_asset("TestAsset", comment="A asset")

# Now lets create and upload a simple asset.
with tempfile.TemporaryDirectory() as tmpdirname:
    assetdir = Path(tmpdirname) / "TestAsset"
    assetdir.mkdir(parents=True, exist_ok=True)
    with builtins.open(assetdir / "test.txt", "w") as fp:
        fp.write("Hi there")
    ml_instance.upload_assets(assetdir)

In [ ]:
# We are going to have three values in our feature, a controlled vocabulary term from the vocabulary FeatureValue, a file asset and
# an integer value which we will call "TestCol"
ml_instance.create_feature("Feature1", "Image",
                                        terms=["FeatureValue"],
                                        assets=[feature_asset],
                                        metadata=[ColumnDefinition(name='TestCol', type=BuiltinTypes.int2)])

[f.name for f in ml_instance.find_features("Image")]

Now we can add some features to our images.  To streamline the creation of new feature, we create a class that is specific to the arguments required to create it.

In [ ]:
TestFeatureClass = ml_instance.feature_record_class("Image", "Feature1")
TestFeatureClass.model_fields

Now using TestFeatureClass, we can create some instances of the feature and add it.  We must have a exeuction_rid in order to define the feature.

In [ ]:
config = ExecutionConfiguration(
    execution=Execution(description="Sample Execution"), 
    workflow=Workflow(
        name="Sample Workflow", 
        url="https://github.com/informatics-isi-edu/deriva-ml/blob/main/pyproject.toml", 
        workflow_type="Sample Workflow"), 
    workflow_terms=[WorkflowTerm(term=Term.workflow, name="Sample Workflow", description="Sample Workflow Example")],
    description="Our Sample Workflow instance")
configuration_record = ml_instance.initialize_execution(config)
execution_rid = configuration_record.execution_rid

with ml_instance.execution(execution_rid=configuration_record) as exec:
    output_dir = ml_instance.execution_assets_path / "testoutput"
    output_dir.mkdir(parents=True, exist_ok=True)
    with open(output_dir / "test.txt", "w+") as f:
        f.write("Hello there\n")
upload_status = ml_instance.upload_execution(coinfiguration=configuration_record)
e = (list(ml_instance.getPath.deriva_ml.Execution.entities().fetch()))[0]

In [ ]:
def strip_system(d):
    return {k:v for k,v in d.items() if k not in ['RCT', 'RMT', 'RCB', 'RMB']}
    
pd.DataFrame([strip_system(i) for i in ml_instance.list_feature("Image", "Feature1")])

In [ ]:
test_catalog.delete_ermrest_catalog(really=True)

In [ ]:
test_catalog.delete_ermrest_catalog(really=True)

In [ ]:
test_catalog.delete_ermrest_catalog(really=True)